In [1]:
import numpy as np
import neurolab as nl
import random
from PIL import Image
import os
import re

In [2]:
def mat2vec(x):
    m = x.shape[0]*x.shape[1]
    tmp1 = np.zeros(m)

    c = 0
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            tmp1[c] = x[i,j]
            c +=1
    return tmp1

In [3]:
def readImg2array(file,size, threshold= 145):
    pilIN = Image.open(file).convert(mode="L")
    pilIN= pilIN.resize(size)
    #pilIN.thumbnail(size,Image.ANTIALIAS)
    imgArray = np.asarray(pilIN,dtype=np.uint8)
    x = np.zeros(imgArray.shape,dtype=float)
    x[imgArray > threshold] = 1
    x[x==0] = -1
    return x

In [4]:
def array2img(data, outFile = None):
    #data is 1 or -1 matrix
    y = np.zeros(data.shape,dtype=np.uint8)
    y[data==1] = 255
    y[data==-1] = 0
    img = Image.fromarray(y,mode="L")
    if outFile is not None:
        img.save(outFile)
    return img

In [5]:
current_path = os.getcwd()
path = current_path+"/train/"
train_paths = []
names = []

for i in os.listdir(path):
    if re.match(r'..jpg',i):
        train_paths.append(path+i)

for path in train_paths:
    name = path.split("/")[-1].split(".")[0]
    names.append(name)

train_paths


['c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/train/A.jpg',
 'c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/train/B.jpg',
 'c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/train/I.jpg',
 'c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/train/T.jpg',
 'c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/train/Y.jpg']

In [21]:
size = (100,100)
threshold = 250
train = []

for path in train_paths:
    x = readImg2array(path,size,threshold)
    x = mat2vec(x)
    train.append(x)

train = np.array(train)

In [7]:
net = nl.net.newhop(train)
#,transf= nl.trans.HardLims()

In [8]:
noise_percent = 0.1
rango = range(0,size[0]*size[1])
noise_level = round((rango[-1]+1)*noise_percent)
test = []

for i in range (len(train)):
    noise = random.sample(rango, noise_level)
    image_noisy = train[i].copy()
    image_noisy[noise] = -image_noisy[noise]
    test.append(image_noisy)
    array2img(image_noisy.reshape(size),outFile = train_paths[i].replace("train","test"))

test = np.array(test)

In [9]:
output_train = net.sim(train)

for i in range(len(output_train)):
    print(names[i], ": ", (output_train[i] == train[i]).all(), 'Sim. steps', len(net.layers[0].outs))
    array2img(output_train[i].reshape(size),outFile = "./output/"+names[i]+"_train.jpg")


A :  False Sim. steps 2
B :  False Sim. steps 2
I :  False Sim. steps 2
T :  False Sim. steps 2
Y :  False Sim. steps 2


In [10]:
output_test = net.sim(test)

for i in range(len(output_train)):
    print(names[i],": ",(output_test[i] == train[i]).all(), 'Sim. steps', len(net.layers[0].outs))
    array2img(output_test[i].reshape(size),outFile = "./output/"+names[i]+"_test.jpg")


A :  False Sim. steps 2
B :  False Sim. steps 2
I :  False Sim. steps 2
T :  False Sim. steps 2
Y :  False Sim. steps 2
